# OpenSearch Warming Up
## Ref: 
- OpenSearch Python Client
    - https://opensearch.org/docs/1.0/clients/python/

In [14]:
from typing import List, Tuple
from opensearchpy import OpenSearch, RequestsHttpConnection


## OpenSearch Client 생성

In [15]:
def create_aws_opensearch_client(region: str, host: str, http_auth: Tuple[str, str]) -> OpenSearch:
    '''
    오픈서치 클라이언트를 제공함.
    '''
    aws_client = OpenSearch(
        hosts = [{'host': host.replace("https://", ""), 'port': 443}],
        http_auth = http_auth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection
    )
    
    return aws_client
    
aws_region = 'eu-west-1'
opensearch_cluster_domain = 'https://search-opensearchservi-njotxrtxhtld-uwpraevjqz3fggqcb44enk3fze.eu-west-1.es.amazonaws.com'
http_auth = ('opensearchuser', 'MarsEarth1!')

aws_client = create_aws_opensearch_client(
                                     aws_region,
                                     opensearch_cluster_domain,
                                     http_auth)
aws_client    

<OpenSearch([{'host': 'search-opensearchservi-njotxrtxhtld-uwpraevjqz3fggqcb44enk3fze.eu-west-1.es.amazonaws.com', 'port': 443}])>

## Index Creation

In [16]:
def create_index(aws_client, index_name, index_body):    
    response = aws_client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)
    
    
# Create an index with non-default settings.
index_name = 'python-test-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 2
    }
  }
}
    
create_index(aws_client, index_name, index_body)    


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test-index'}


## Check if index exists

In [2]:
def check_if_index_exists(aws_client, index_name):
    '''
    인덱스가 존재하는지 확인
    '''
    exists = aws_client.indices.exists(index_name)
    print(f"index_name={index_name}, exists={exists}")
    return exists


index_exists = check_if_index_exists(aws_client, index_name)
index_exists

NameError: name 'aws_client' is not defined

## Add Documents

In [18]:
# Add a document to the index.
def add_doc(aws_client, index_name, document, id):
    response = aws_client.index(
        index = index_name,
        body = document,
        id = id,
        refresh = True
    )

    print('\nAdding document:')
    print(response)
    
document = {
  'title': 'Moneyball',
  'director': 'Bennett Miller',
  'year': '2011'
}
id = '1'

add_doc(aws_client, index_name, document, id)

document = {
  'title': 'Parasite',
  'director': 'Junho Bong',
  'year': '2019'
}
id = '2'

add_doc(aws_client, index_name, document, id)


Adding document:
{'_index': 'python-test-index', '_id': '1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}

Adding document:
{'_index': 'python-test-index', '_id': '2', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


## Search Document

In [19]:
# Search for the document.
def search_document(aws_client, query, index_name):
    response = aws_client.search(
        body = query,
        index = index_name
    )
    print('\nSearch results:')
    print(response)
    
q = 'miller'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title', 'director']
    }
  }
}
    
search_document(aws_client, query, index_name)    


Search results:
{'took': 3, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.6931471, 'hits': [{'_index': 'python-test-index', '_id': '1', '_score': 0.6931471, '_source': {'title': 'Moneyball', 'director': 'Bennett Miller', 'year': '2011'}}]}}


In [20]:
# Delete the document.

def delete_doc(aws_client, index_name, id):
    response = aws_client.delete(
        index = index_name,
        id = id
    )

    print('\nDeleting document:')
    print(response)
    
id = 1    
delete_doc(aws_client, index_name, id)    


Deleting document:
{'_index': 'python-test-index', '_id': '1', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}


In [21]:
# Delete the index.

def delete_index(aws_client, index_name):
    response = aws_client.indices.delete(
        index = index_name
    )

    print('\nDeleting index:')
    print(response)
    
delete_index(aws_client, index_name)    


Deleting index:
{'acknowledged': True}
